In [49]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
from random import randint

from model import Model
from load_data import load_data

In [2]:
train_dataLoader, test_dataLoader, = load_data("data3/normal_vpm.json",batch_size=128,test_size=0.1)

In [3]:
model1 = Model()
optimizer = optim.Adam(model1.parameters(),lr=0.001)
criterion = nn.MSELoss()

In [4]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model1.to(device)
print("Running model on:",device)

Running model on: cpu


In [6]:
model1.fit(train_dataLoader,optimizer,criterion,epochs=50)

Losses:   0%|          | 0/50 [00:00<?, ?it/s]

In [20]:
model1.test(test_dataLoader,treshold=0.5)


Corectly predicted: 1797 out of 2034
Test accuracy: 88.35%


Przykładowa wartość dla której nie ma zakupu, zobaczymy czy dzięki naszemu systemowi byli byśmy wstanie przekonać do zakupu
{"d_time":0.100068132,"n_visits":2,"visits_per_minute":0.0001321401,"max_disc":0,"curr_disc":0,"is_purchased":-1}

In [14]:
def best_discount1(discount_list):
    return discount_list.index(max(discount_list)) * 5


In [17]:
def best_discount2(discount_list,threshold):
    for i,disc in enumerate(discount_list):
        if disc > threshold:
            return i * 5
    return len(discount_list) * 5

In [78]:
def best_discount3(discount_list):
    if max(discount_list) <= 0:
        return 20
    while True:
        idx = randint(0,4)
        if discount_list[idx] > 0:
            return idx * 5
        

In [88]:
def get_discounts(dataloader):
    out = []
    bought = 0
    for X,Y in dataloader:
        for y in Y:
            if y > 0:
                bought +=1
        for x in X:
            x_ = x.squeeze().tolist()[:4]
            inp = torch.Tensor([x_ + [d*5] for d in range(5)])
            out.append(model1(inp).squeeze().tolist())
    return out, bought

In [18]:
model1.cpu()
inp = torch.Tensor([[0.100068132, 2, 0.0001321401, 0, x*5] for x in range(10)])
out = model1(inp).squeeze().tolist()
print(out)
for prob,disc in zip(out,[5*x for x in range(10)]):
    print(f'FOR {disc}% discount probablity of buying is {prob:.5}')
#print(model1(torch.Tensor(np.array([60000000,2,0,0]))))
print(f'BEST discount1={best_discount1(out)}')
print(f'BEST discount2={best_discount2(out,0.8)}')

[-0.8980417251586914, 0.9287691712379456, 0.9997364282608032, 0.999998927116394, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
FOR 0% discount probablity of buying is -0.89804
FOR 5% discount probablity of buying is 0.92877
FOR 10% discount probablity of buying is 0.99974
FOR 15% discount probablity of buying is 1.0
FOR 20% discount probablity of buying is 1.0
FOR 25% discount probablity of buying is 1.0
FOR 30% discount probablity of buying is 1.0
FOR 35% discount probablity of buying is 1.0
FOR 40% discount probablity of buying is 1.0
FOR 45% discount probablity of buying is 1.0
BEST discount1=20
BEST discount2=5


In [95]:
list1 = []
list2 = []
list3 = []
disc_list, bought = get_discounts(test_dataLoader)
for disc in disc_list:
    list1.append(best_discount1(disc))
    list2.append(best_discount2(disc,0.8))
    list3.append(best_discount3(disc))
print(f'BOUGHT: {bought} out of {len(test_dataLoader.dataset)}')
print(np.average(list1))
print(np.average(list2))
print(np.average(list3))


BOUGHT: 927 out of 2034
17.77777777777778
12.372173058013766
14.601769911504425
